In [3]:
import numpy as np
import pandas as pd
import dat
from collections import defaultdict
import json
import re
import os
import mysql.connector
from datetime import datetime, timedelta, date

In [ ]:
class mysql_profiler:
    '''Class to create quick profiling of tables located on mysql server'''

    def __init__(self,host,user,password,database,df) -> None:
        self.df = df
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        
    def multiple_dataset_apply_mysql(host:str, user:str, password:str, database:str):
        ''' This function reads multiple tables from connected database as parameter 
        then runs all functions on dat package to read table '''

        #Regex pattern for date columns
        pattern_d = re.compile(r"[0-9]{4}.[0-9]{2}.[0-9]{2}.*", re.IGNORECASE)
        pattern_d_alt = re.compile(r"[0-9]{2}.[0-9]{2}.[0-9]{4}.*", re.IGNORECASE)

        # Creating connection string
        db=mysql.connector.connect(host = str(host), user = str(user), password = str(password), database = str(database))
        cursor=db.cursor()
        cursor.execute("SHOW TABLES")
        myresult = cursor.fetchall()

        # saving all tables of database into a list
        read_table_names = [table_name[0] for table_name in myresult]

        # Dict for dataframes that has read from database
        dataframes_dict = {}

        # Creating working directory for daily partitioning
        dir = os.path.join("C:\\", "Users\Lenovo\Desktop\exam_eti\containerized_tool\data_analysis_tool", f'{date.today()}')
        if not os.path.exists(dir):
            os.mkdir(dir)
        os.chdir(dir)

        # loop over the list of sql tables and read them into pandas dataframe
        for f in read_table_names:
            df = pd.read_sql(f'SELECT * FROM {f}', con=db)
            # Creating a dict consisting of all dataframes
            if df.size != 0:
                dataframes_dict[f] = df  
            # Applying methods to dataframes defined on analysis_dict.py file
            for key,value in dat.analysis_dict().items():
                if df.size != 0:
                    vars()[key] = value(df)
                    dat.save_dataframe_excel(vars(),f"analysis_{f}_{date.today()}")
                else:
                    dat.save_dataframe_excel(df,f"analysis_{f}_{date.today()}")
        return "methods applied xlsx file saved",dataframes_dict